In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [55]:
import os
import numpy as np
import pandas as pd
import torch
from multiprocessing import Pool
from sklearn.preprocessing import MinMaxScaler
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import joblib
import os
import numpy as np
import pandas as pd
import torch
from multiprocessing import Pool
from sklearn.preprocessing import MinMaxScaler
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import joblib

SEQUENCE_LENGTH = 600
BATCH_SIZE = 64
EPOCHS = 250
LEARNING_RATE = 1e-5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!unzip -qq '/content/gdrive/MyDrive/data_aug_new.zip'

#Load file

In [59]:
data = np.load('/content/gdrive/MyDrive/qani_train600.npz')
data1 = np.load('/content/gdrive/MyDrive/qani_valid600.npz')

In [65]:



X_original = data['X_original']
y_original = data['y_original']
X_augmented = data['X_augmented']
y_augmented = data['y_augmented']


X_original1 = data1['X_original']
y_original1 = data1['y_original']
X_augmented1 = data1['X_augmented']
y_augmented1 = data1['y_augmented']


# Convert data to PyTorch tensors
try:
    X_original = torch.tensor(np.array(X_original), dtype=torch.float32).to(DEVICE)
    y_original = torch.tensor(np.array(y_original), dtype=torch.float32).to(DEVICE)
    X_augmented = torch.tensor(np.array(X_augmented), dtype=torch.float32).to(DEVICE)
    y_augmented = torch.tensor(np.array(y_augmented), dtype=torch.float32).to(DEVICE)
except Exception as e:
    print(f"Error during tensor conversion: {e}")
    print(f"Shapes: X_original - {np.array(X_original).shape}, y_original - {np.array(y_original).shape}")
    print(f"Shapes: X_augmented - {np.array(X_augmented).shape}, y_augmented - {np.array(y_augmented).shape}")
    raise


# Convert data to PyTorch tensors
try:
    X_original1 = torch.tensor(np.array(X_original1), dtype=torch.float32).to(DEVICE)
    y_original1 = torch.tensor(np.array(y_original1), dtype=torch.float32).to(DEVICE)
    X_augmented1 = torch.tensor(np.array(X_augmented1), dtype=torch.float32).to(DEVICE)
    y_augmented1 = torch.tensor(np.array(y_augmented1), dtype=torch.float32).to(DEVICE)
except Exception as e:
    print(f"Error during tensor conversion: {e}")
    print(f"Shapes: X_original1 - {np.array(X_original1).shape}, y_original1 - {np.array(y_original1).shape}")
    print(f"Shapes: X_augmented1 - {np.array(X_augmented1).shape}, y_augmented1 - {np.array(y_augmented1).shape}")
    raise


X_train = torch.cat([X_original, X_augmented])
y_train = torch.cat([y_original, y_augmented])

X_train1 = torch.cat([X_original1, X_augmented1])
y_train1 = torch.cat([y_original1, y_augmented1])

In [58]:
# this cell for delete coolant temp from data
X_train = X_train[:, :, :-1]  # Slicing to exclude the last column
X_train1 = X_train1[:, :, :-1]  # Slicing to exclude the last column


# **Normalize**

In [66]:

# Convert the min and max values to tensors
min_val_x = torch.tensor([0, 0, -10, -10,0], dtype=torch.float32).to(DEVICE)
max_val_x = torch.tensor([6, 150, 10, 10,130], dtype=torch.float32).to(DEVICE)

#for without coolant
# min_val_x = torch.tensor([0, 0, -10, -10], dtype=torch.float32).to(DEVICE)
# max_val_x = torch.tensor([6, 150, 10, 10], dtype=torch.float32).to(DEVICE)


min_val_y = torch.tensor([0], dtype=torch.float32).to(DEVICE)
max_val_y = torch.tensor([10000], dtype=torch.float32).to(DEVICE)

# Custom normalization function for X
def custom_normalize_X(data, min_vals, max_vals):
    for i in range(data.shape[-1]):
        data[:, :, i] = (data[:, :, i] - min_vals[i]) / (max_vals[i] - min_vals[i])
    return data

# Custom normalization function for y
def custom_normalize_y(data, min_val, max_val):
    return (data - min_val) / (max_val - min_val)

# Normalize X_train and X_test
X_train_normalized = custom_normalize_X(X_train, min_val_x, max_val_x)
# X_test_normalized = custom_normalize_X(X_test, min_val_x, max_val_x)

# Normalize y_train and y_test
y_train_normalized = custom_normalize_y(y_train, min_val_y, max_val_y)
# y_test_normalized = custom_normalize_y(y_test, min_val_y, max_val_y)


# Normalize X_train and X_test
X_train_normalized1 = custom_normalize_X(X_train1, min_val_x, max_val_x)
# X_test_normalized = custom_normalize_X(X_test, min_val_x, max_val_x)

# Normalize y_train and y_test
y_train_normalized1 = custom_normalize_y(y_train1, min_val_y, max_val_y)
# y_test_normalized = custom_normalize_y(y_test, min_val_y, max_val_y)


# **Standard**

In [35]:
# Compute the mean and standard deviation for X and y from the training set
mean_x = torch.mean(X_train, dim=(0, 1)).to(DEVICE)
std_x = torch.std(X_train, dim=(0, 1)).to(DEVICE)

mean_y = torch.mean(y_train).to(DEVICE)
std_y = torch.std(y_train).to(DEVICE)

# Custom standardization function for X
def custom_standardize_X(data, mean_vals, std_vals):
    for i in range(data.shape[-1]):
        data[:, :, i] = (data[:, :, i] - mean_vals[i]) / std_vals[i]
    return data

# Custom standardization function for y
def custom_standardize_y(data, mean_val, std_val):
    return (data - mean_val) / std_val

# Standardize X_train and y_train
X_train_normalized = custom_standardize_X(X_train, mean_x, std_x)
y_train_normalized = custom_standardize_y(y_train, mean_y, std_y)

# # Standardize X_test and y_test using the same mean and std as training data
# X_test_standardized = custom_standardize_X(X_test, mean_x, std_x)
# y_test_standardized = custom_standardize_y(y_test, mean_y, std_y)


X_train_normalized1 = custom_standardize_X(X_train1, mean_x, std_x)
y_train_normalized1 = custom_standardize_y(y_train1, mean_y, std_y)


import json

# Convert tensors to lists to make them serializable
mean_x_list = mean_x.cpu().tolist()
std_x_list = std_x.cpu().tolist()
mean_y_list = mean_y.cpu().tolist()
std_y_list = std_y.cpu().tolist()

# Save the mean and std values in a JSON file
normalization_params = {
    'mean_x': mean_x_list,
    'std_x': std_x_list,
    'mean_y': mean_y_list,
    'std_y': std_y_list
}

# Write the normalization parameters to a JSON file
with open('normalization_params.json', 'w') as f:
    json.dump(normalization_params, f)


# **model**

In [67]:
import torch.nn as nn
import torch

class FuelConsumptionModel(nn.Module):
    def __init__(self, input_size):
        super(FuelConsumptionModel, self).__init__()
        # LSTM layers
        self.lstm1 = nn.LSTM(input_size, 32, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(64, 32, batch_first=True, bidirectional=True)

        # Layer Normalization after each LSTM
        self.layer_norm1 = nn.LayerNorm(64)
        self.layer_norm2 = nn.LayerNorm(64)

        # Dropout layers
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.2)

        # Dense output layer
        self.dense = nn.Linear(64, 1)

    def forward(self, x):
        # LSTM 1 + Layer Normalization + Dropout
        x, _ = self.lstm1(x)
        x = self.layer_norm1(x)  # Apply layer normalization after LSTM1
        x = self.dropout1(x)

        # LSTM 2 + Layer Normalization + Dropout
        x, _ = self.lstm2(x)
        x = self.layer_norm2(x)  # Apply layer normalization after LSTM2
        x = self.dropout2(x)

        # Dense layer for final output
        x = self.dense(x)

        return x


In [68]:
# Check if X_train_normalized and y_train_normalized have the same number of samples
print(f"X_train_normalized shape: {X_train_normalized.shape}")
print(f"y_train_normalized shape: {y_train_normalized.shape}")

# Ensure they have the same length in the first dimension
if len(X_train_normalized) != len(y_train_normalized):
    raise ValueError(f"Mismatch in number of samples: {len(X_train_normalized)} in X vs {len(y_train_normalized)} in y")

# Calculate train size
train_size = int(0.8 * len(X_train_normalized))

# Split the data while preserving the order
X_train_split = X_train_normalized[:train_size]
y_train_split = y_train_normalized[:train_size]

X_val_split = X_train_normalized[train_size:]
y_val_split = y_train_normalized[train_size:]


# X_train_split = X_train_normalized
# y_train_split = y_train_normalized

# X_val_split = X_train_normalized1
# y_val_split = y_train_normalized1

print(f"Train size: {train_size}, Validation size: {len(X_val_split)}")


X_train_normalized shape: torch.Size([69439, 600, 5])
y_train_normalized shape: torch.Size([69439, 600, 1])
Train size: 55551, Validation size: 13888


# **train with coolant**

In [69]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR  # Importing StepLR for learning rate decay
from torch.optim.lr_scheduler import ReduceLROnPlateau
# Assuming `FuelConsumptionModel`, `X_train_normalized`, `train_loader`, `val_loader`, `DEVICE`, and `EPOCHS` are defined

# Choose a single loss function to use for all experiments
criterion = nn.L1Loss()  # You can change this to your preferred loss function

EPOCHS = 250
# Define weight decay values and initial learning rates for decay schedules
weight_decay_values = 1e-5
initial_learning_rates = [1e-4]  # Two starting learning rates

# Initialize variables to track the best parameters
best_weight_decay = None
best_initial_lr = None
best_model_weights = None
best_val_loss = float('inf')

ini_batches = [64]
# Loop over all combinations of initial learning rates and weight decay values

BATCH_SIZE = 64
# Create DataLoader for training and validation sets
train_loader = DataLoader(TensorDataset(X_train_split, y_train_split), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_split, y_val_split), batch_size=BATCH_SIZE, shuffle=False)

for init_lr in initial_learning_rates:
    # Initialize model, criterion, and optimizer for each combination
    model = FuelConsumptionModel(input_size=X_train_normalized.shape[-1]).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=init_lr, weight_decay=weight_decay_values)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    # scheduler = StepLR(optimizer, step_size=50, gamma=0.5)  # Decay LR by 0.5 every 50 epochs

    # Early stopping parameters
    patience = 7  # Number of epochs to wait before stopping if no improvement
    best_loss = float('inf')  # Initialize best loss to infinity
    epochs_without_improvement = 0  # Counter for epochs without improvement

    # Training loop with early stopping
    model.train()
    for epoch in range(EPOCHS):
        running_loss = 0.0
        model.train()  # Ensure model is in training mode

        # Training phase
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)  # Ensure data is on the correct device
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Calculate average loss for the epoch
        avg_training_loss = running_loss / len(train_loader)

        # Validation phase
        model.eval()  # Switch to evaluation mode
        val_running_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:  # Assume you have a validation DataLoader `val_loader`
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)  # Ensure data is on the correct device
                outputs = model(inputs)
                val_loss = criterion(outputs, targets)
                val_running_loss += val_loss.item()

        avg_val_loss = val_running_loss / len(val_loader)

        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch [{epoch+1}/{EPOCHS}] ---- Training Loss: {avg_training_loss:.4f} ---- Validation Loss: {avg_val_loss:.6f} ------{current_lr}")

        # print(f"Epoch [{epoch+1}/{EPOCHS}] ---- Training Loss: {avg_training_loss:.4f} ---- Validation Loss: {avg_val_loss:.4f}    Init LR: {init_lr:.0e}  WD: {wd:.0e}")

        # Adjust the learning rate
        scheduler.step(avg_val_loss)
        # scheduler.step()

        # Early stopping check
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        # Save the model for the current combination
        model_filename = f'best_fuel_consumption_model.pth'
        torch.save(model.state_dict(), model_filename)


print(f"Saved model: {model_filename}")

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/250] ---- Training Loss: 0.0773 ---- Validation Loss: 0.030268 ------0.0001
Epoch [2/250] ---- Training Loss: 0.0314 ---- Validation Loss: 0.025982 ------0.0001
Epoch [3/250] ---- Training Loss: 0.0262 ---- Validation Loss: 0.021106 ------0.0001
Epoch [4/250] ---- Training Loss: 0.0214 ---- Validation Loss: 0.017569 ------0.0001
Epoch [5/250] ---- Training Loss: 0.0187 ---- Validation Loss: 0.016428 ------0.0001
Epoch [6/250] ---- Training Loss: 0.0175 ---- Validation Loss: 0.015582 ------0.0001
Epoch [7/250] ---- Training Loss: 0.0169 ---- Validation Loss: 0.015281 ------0.0001
Epoch [8/250] ---- Training Loss: 0.0165 ---- Validation Loss: 0.015198 ------0.0001
Epoch [9/250] ---- Training Loss: 0.0162 ---- Validation Loss: 0.014927 ------0.0001
Epoch [10/250] ---- Training Loss: 0.0159 ---- Validation Loss: 0.014752 ------0.0001
Epoch [11/250] ---- Training Loss: 0.0157 ---- Validation Loss: 0.014699 ------0.0001
Epoch [12/250] ---- Training Loss: 0.0155 ---- Validation Loss:

# **train without coolant**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR  # Importing StepLR for learning rate decay
from torch.optim.lr_scheduler import ReduceLROnPlateau
# Assuming `FuelConsumptionModel`, `X_train_normalized`, `train_loader`, `val_loader`, `DEVICE`, and `EPOCHS` are defined

# Choose a single loss function to use for all experiments
criterion = nn.L1Loss()  # You can change this to your preferred loss function

EPOCHS = 500
# Define weight decay values and initial learning rates for decay schedules
weight_decay_values = 1e-6
initial_learning_rates = [1e-4]  # Two starting learning rates

# Initialize variables to track the best parameters
best_weight_decay = None
best_initial_lr = None
best_model_weights = None
best_val_loss = float('inf')

ini_batches = [64]
# Loop over all combinations of initial learning rates and weight decay values

BATCH_SIZE = 64
# Create DataLoader for training and validation sets
train_loader = DataLoader(TensorDataset(X_train_split, y_train_split), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_split, y_val_split), batch_size=BATCH_SIZE, shuffle=False)

for init_lr in initial_learning_rates:
    # Initialize model, criterion, and optimizer for each combination
    model = FuelConsumptionModel(input_size=X_train_normalized.shape[-1]).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=init_lr, weight_decay=weight_decay_values)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
    # Early stopping parameters
    patience = 20  # Number of epochs to wait before stopping if no improvement
    best_loss = float('inf')  # Initialize best loss to infinity
    epochs_without_improvement = 0  # Counter for epochs without improvement

    # Training loop with early stopping
    model.train()
    for epoch in range(EPOCHS):
        running_loss = 0.0
        model.train()  # Ensure model is in training mode

        # Training phase
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)  # Ensure data is on the correct device
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Calculate average loss for the epoch
        avg_training_loss = running_loss / len(train_loader)

        # Validation phase
        model.eval()  # Switch to evaluation mode
        val_running_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:  # Assume you have a validation DataLoader `val_loader`
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)  # Ensure data is on the correct device
                outputs = model(inputs)
                val_loss = criterion(outputs, targets)
                val_running_loss += val_loss.item()

        avg_val_loss = val_running_loss / len(val_loader)

        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch [{epoch+1}/{EPOCHS}] ---- Training Loss: {avg_training_loss:.4f} ---- Validation Loss: {avg_val_loss:.6f} ------{current_lr}")

        # print(f"Epoch [{epoch+1}/{EPOCHS}] ---- Training Loss: {avg_training_loss:.4f} ---- Validation Loss: {avg_val_loss:.4f}    Init LR: {init_lr:.0e}  WD: {wd:.0e}")

        # Adjust the learning rate
        scheduler.step(avg_val_loss)

        # Early stopping check
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        # Save the model for the current combination
        model_filename = f'mode_without_coolant_qani.pth'
        torch.save(model.state_dict(), model_filename)


print(f"Saved model: {model_filename}")

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/500] ---- Training Loss: 0.0782 ---- Validation Loss: 0.027671 ------0.0001
Epoch [2/500] ---- Training Loss: 0.0281 ---- Validation Loss: 0.023100 ------0.0001
Epoch [3/500] ---- Training Loss: 0.0233 ---- Validation Loss: 0.019051 ------0.0001
Epoch [4/500] ---- Training Loss: 0.0201 ---- Validation Loss: 0.016721 ------0.0001
Epoch [5/500] ---- Training Loss: 0.0182 ---- Validation Loss: 0.015666 ------0.0001
Epoch [6/500] ---- Training Loss: 0.0172 ---- Validation Loss: 0.015233 ------0.0001
Epoch [7/500] ---- Training Loss: 0.0166 ---- Validation Loss: 0.014904 ------0.0001
Epoch [8/500] ---- Training Loss: 0.0161 ---- Validation Loss: 0.014731 ------0.0001
Epoch [9/500] ---- Training Loss: 0.0158 ---- Validation Loss: 0.014515 ------0.0001
Epoch [10/500] ---- Training Loss: 0.0155 ---- Validation Loss: 0.014424 ------0.0001
Epoch [11/500] ---- Training Loss: 0.0153 ---- Validation Loss: 0.014314 ------0.0001
Epoch [12/500] ---- Training Loss: 0.0151 ---- Validation Loss:

#Test with all saved model


In [70]:
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn as nn
import torch

# Constants
SEQUENCE_LENGTH = 600  # Updated sequence length
PLOT_SAVE_DIR = 'predicted_vs_actual_plots'  # Base directory to save plots and CSVs
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure the base save directory exists
os.makedirs(PLOT_SAVE_DIR, exist_ok=True)

# Define the PyTorch model structure (same as the one used for training)


# Load the trained model
def load_trained_model(model_path, input_size):
    model = FuelConsumptionModel(input_size=input_size)
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()
    return model

# Process the file and prepare segments
def process_file(file_path):
    df = pd.read_csv(file_path)
    # df = df.iloc[::2].reset_index(drop=True)
    df['Time'] = df['time'] - df['time'].iloc[0]
    df['Trip fuel consumption'] = df['Trip_fuel_consumption'] - df['Trip_fuel_consumption'].iloc[0]
    df['Acceleration'] = df['Vehicle_Speed'].diff().fillna(0)
    df['Momentary fuel consumption'] = df['Trip fuel consumption'].diff().fillna(0)

    df['Adjusted_gear_position'] = df['Current_gear_shift_position_(Current_gear)'].replace({13: 0, 14: 1})

    # Selecting features and target
    features = df[['Adjusted_gear_position', 'Vehicle_Speed', 'slope', 'Acceleration','Coolant_temperature']]
    # features = df[['Adjusted_gear_position', 'Vehicle_Speed', 'slope', 'Acceleration']]

    target = df['Momentary fuel consumption']
    target = df['Momentary fuel consumption']
    return features, target

# Pad and normalize the data
def pad_and_normalize(data, sequence_length=SEQUENCE_LENGTH):
    padded_data = np.zeros((len(data), sequence_length, data[0].shape[1]))
    for i, seq in enumerate(data):
        length = min(len(seq), sequence_length)
        padded_data[i, :length] = seq[:length]

    # Normalization (same as in your script)
    min_val_x = [0, 0, -10, -10,0]
    max_val_x = [6, 150, 10, 10,130]
    for i in range(padded_data.shape[-1]):
        padded_data[:, :, i] = (padded_data[:, :, i] - min_val_x[i]) / (max_val_x[i] - min_val_x[i])

    return torch.tensor(padded_data, dtype=torch.float32).to(DEVICE)




# Predict and plot the results
def plot_predicted_vs_real(input_file, model, model_name):
    features, actual_values = process_file(input_file)
    num_segments = len(features) // SEQUENCE_LENGTH
    predictions = []

    for i in range(num_segments):
        segment = features.iloc[i * SEQUENCE_LENGTH:(i + 1) * SEQUENCE_LENGTH]
        segment_normalized = pad_and_normalize([segment.values])
        with torch.no_grad():
            segment_predictions = model(segment_normalized).cpu().numpy()
        predictions.extend(segment_predictions.flatten() * 10000)

    # Handle any remaining data
    remainder = len(features) % SEQUENCE_LENGTH
    if remainder != 0:
        last_segment = features.iloc[-remainder:]
        last_segment_normalized = pad_and_normalize([last_segment.values], sequence_length=remainder)
        with torch.no_grad():
            last_segment_predictions = model(last_segment_normalized).cpu().numpy()
        predictions.extend(last_segment_predictions.flatten() * 10000)

    predictions = np.array(predictions)
    actual_valuess = actual_values.values[:len(predictions)]  # Ensure lengths match

    # Calculate error metrics
    mae = mean_absolute_error(actual_valuess, predictions)
    mse = mean_squared_error(actual_valuess, predictions)

    # Handling MAPE: Ignore zero actual values to avoid NaN issues
    non_zero_actual = actual_valuess != 0
    mape = np.mean(np.abs((actual_valuess[non_zero_actual] - predictions[non_zero_actual]) / actual_valuess[non_zero_actual])) * 100

    print(f'MAE: {mae:.4f}---------------{input_file}')
    print(f'MSE: {mse:.4f}')
    print(f'MAPE: {mape:.4f}% ---------------------')


    plt.figure(figsize=(10, 6))
    plt.plot(np.cumsum(actual_values.values[:len(predictions)], axis=0), label='Real', color='blue')
    plt.plot(np.cumsum(predictions[:len(actual_values)], axis=0), label='Predicted', color='red')
    plt.xlabel('Index')
    plt.ylabel('Fuel Consumption')
    plt.title(f'Predicted vs Real Fuel Consumption ({model_name})')
    plt.legend()

    # Create model-specific directory
    model_save_dir = os.path.join(PLOT_SAVE_DIR, model_name)
    os.makedirs(model_save_dir, exist_ok=True)

    # Save plot using model name in the designated directory
    plot_filename = os.path.join(model_save_dir, f'{os.path.splitext(os.path.basename(input_file))[0]}_all.png')
    plt.savefig(plot_filename)
    plt.close()

    # Save predictions and actual values to CSV using model name in the designated directory
    results_df = pd.DataFrame({
        'Speed': features["Vehicle_Speed"].iloc[:len(predictions)],
        'Actual': np.cumsum(actual_values.values[:len(predictions)], axis=0),
        'Predicted': np.cumsum(predictions[:len(actual_values)], axis=0),
        'MAE': mae,  # Add MAE to the dataframe
        'MSE': mse,  # Add MSE to the dataframe
    })

    csv_filename = os.path.join(model_save_dir, f'{os.path.splitext(os.path.basename(input_file))[0]}.csv')
    results_df.to_csv(csv_filename, index=False)
    print(f"CSV saved as: {csv_filename}")



# Directory containing all saved models and CSV files
csv_dir = '/content/'  # Directory containing CSV files
model_dir = '/content/'  # Directory containing model files

# List all model files in the directory
model_files = [f for f in os.listdir(model_dir) if f.endswith('.pth')]

# List all CSV files in the directory
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

# Load and test each model on each CSV file
input_size = 4  # Number of features in the input data
input_size = 5  # Number of features in the input data


for csv_file in csv_files:
    csv_file_path = os.path.join(csv_dir, csv_file)
    # print(f"Processing CSV file: {csv_file}")

    for model_file in model_files:
        model_path = os.path.join(model_dir, model_file)
        model_name = os.path.splitext(model_file)[0]  # Get the base name of the model file
        # print(f"Testing model: {model_name} on {csv_file}")

        model = load_trained_model(model_path, input_size)
        plot_predicted_vs_real(csv_file_path, model, model_name)


<ipython-input-70-313dd3c67ca7>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


MAE: 101.8331---------------/content/Trip_7.csv
MSE: 26029.5321
MAPE: 33.5239% ---------------------
CSV saved as: predicted_vs_actual_plots/old_normal_withcool/Trip_7.csv
MAE: 112.3577---------------/content/Trip_11.csv
MSE: 26301.1429
MAPE: 34.7410% ---------------------


<ipython-input-70-313dd3c67ca7>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


CSV saved as: predicted_vs_actual_plots/old_normal_withcool/Trip_11.csv
MAE: 85.0616---------------/content/Trip_14.csv
MSE: 19141.1750
MAPE: 32.3620% ---------------------


<ipython-input-70-313dd3c67ca7>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


...

CSV saved as: predicted_vs_actual_plots/old_normal_withcool/Trip_10.csv
MAE: 109.0339---------------/content/Trip_9.csv
MSE: 27923.0890
MAPE: 28.8298% ---------------------


<ipython-input-70-313dd3c67ca7>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


CSV saved as: predicted_vs_actual_plots/old_normal_withcool/Trip_9.csv
MAE: 111.7906---------------/content/Trip_1.csv
MSE: 26248.2288
MAPE: 31.9066% ---------------------


<ipython-input-70-313dd3c67ca7>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


CSV saved as: predicted_vs_actual_plots/old_normal_withcool/Trip_1.csv


# **test with standard**

In [43]:
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn as nn
import torch

# Constants
SEQUENCE_LENGTH = 600  # Updated sequence length
PLOT_SAVE_DIR = 'predicted_vs_actual_plots'  # Base directory to save plots and CSVs
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure the base save directory exists
os.makedirs(PLOT_SAVE_DIR, exist_ok=True)

# Define the PyTorch model structure (same as the one used for training)


# Load the trained model
def load_trained_model(model_path, input_size):
    model = FuelConsumptionModel(input_size=input_size)
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()
    return model


# Load standardization parameters from JSON
def load_normalization_params(json_file):
    with open(json_file, 'r') as f:
        params = json.load(f)
    mean_x = torch.tensor(params['mean_x'], dtype=torch.float32).to(DEVICE)
    std_x = torch.tensor(params['std_x'], dtype=torch.float32).to(DEVICE)
    mean_y = torch.tensor(params['mean_y'], dtype=torch.float32).to(DEVICE)
    std_y = torch.tensor(params['std_y'], dtype=torch.float32).to(DEVICE)
    return mean_x, std_x, mean_y, std_y

# Custom standardization function for X
def custom_standardize_X(data, mean_vals, std_vals):
    for i in range(data.shape[-1]):
        data[:, :, i] = (data[:, :, i] - mean_vals[i]) / std_vals[i]
    return data

# Custom standardization function for y
def custom_standardize_y(data, mean_val, std_val):
    return (data - mean_val) / std_val

# Process the file and standardize the data
def process_file(file_path, mean_x, std_x, mean_y, std_y):
    df = pd.read_csv(file_path)
    df['Time'] = df['time'] - df['time'].iloc[0]
    df['Trip fuel consumption'] = df['Trip_fuel_consumption'] - df['Trip_fuel_consumption'].iloc[0]
    df['Acceleration'] = df['Vehicle_Speed'].diff().fillna(0)
    df['Momentary fuel consumption'] = df['Trip fuel consumption'].diff().fillna(0)
    df['Adjusted_gear_position'] = df['Current_gear_shift_position_(Current_gear)'].replace({13: 0, 14: 1})

    # Selecting features and target
    features = df[['Adjusted_gear_position', 'Vehicle_Speed', 'slope', 'Acceleration','Coolant_temperature']]
    features = df[['Adjusted_gear_position', 'Vehicle_Speed', 'slope', 'Acceleration']]

    target = df['Momentary fuel consumption']

    # Convert features and target to tensors
    features = torch.tensor(features.values, dtype=torch.float32).to(DEVICE)
    target = torch.tensor(target.values, dtype=torch.float32).to(DEVICE)

    # Apply standardization
    features = custom_standardize_X(features.unsqueeze(0), mean_x, std_x).squeeze(0)
    # target = custom_standardize_y(target, mean_y, std_y)

    return features, target

# Plot and predict function updated to handle standardization
def plot_predicted_vs_real(input_file, model, model_name, mean_x, std_x, mean_y, std_y):
    features, actual_values = process_file(input_file, mean_x, std_x, mean_y, std_y)
    num_segments = len(features) // SEQUENCE_LENGTH
    predictions = []

    for i in range(num_segments):
        segment = features[i * SEQUENCE_LENGTH:(i + 1) * SEQUENCE_LENGTH]
        segment = segment.unsqueeze(0).to(DEVICE)  # Add batch dimension
        with torch.no_grad():
            segment_predictions = model(segment).cpu().numpy()
        predictions.extend(segment_predictions.flatten() * std_y.cpu().numpy() + mean_y.cpu().numpy())  # De-standardize

    remainder = len(features) % SEQUENCE_LENGTH
    if remainder != 0:
        last_segment = features[-remainder:]
        last_segment = last_segment.unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            last_segment_predictions = model(last_segment).cpu().numpy()
        predictions.extend(last_segment_predictions.flatten() * std_y.cpu().numpy() + mean_y.cpu().numpy())  # De-standardize

    predictions = np.array(predictions)
    actual_values = actual_values.cpu().numpy()

    # Calculate error metrics
    mae = mean_absolute_error(actual_values[:len(predictions)], predictions)
    mse = mean_squared_error(actual_values[:len(predictions)], predictions)
    mape = np.mean(np.abs((actual_values[actual_values != 0] - predictions[actual_values != 0]) / actual_values[actual_values != 0])) * 100

    print(f'MAE: {mae:.4f}')
    print(f'MSE: {mse:.4f}')
    print(f'MAPE: {mape:.4f}%')

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(np.cumsum(actual_values[:len(predictions)], axis=0), label='Real', color='blue')
    plt.plot(np.cumsum(predictions, axis=0), label='Predicted', color='red')
    plt.xlabel('Index')
    plt.ylabel('Fuel Consumption')
    plt.title(f'Predicted vs Real Fuel Consumption ({model_name})')
    plt.legend()

    # Save plot and CSV
    model_save_dir = os.path.join(PLOT_SAVE_DIR, model_name)
    os.makedirs(model_save_dir, exist_ok=True)
    plot_filename = os.path.join(model_save_dir, f'{os.path.splitext(os.path.basename(input_file))[0]}_all.png')
    plt.savefig(plot_filename)
    plt.close()

    # Save predictions and actual values to CSV
    results_df = pd.DataFrame({

        'Actual': np.cumsum(actual_values[:len(predictions)], axis=0),
        'Predicted': np.cumsum(predictions, axis=0),
        'MAE': mae,
        'MSE': mse,
    })
    csv_filename = os.path.join(model_save_dir, f'{os.path.splitext(os.path.basename(input_file))[0]}.csv')
    results_df.to_csv(csv_filename, index=False)
    print(f"CSV saved as: {csv_filename}")

# Load normalization parameters
mean_x, std_x, mean_y, std_y = load_normalization_params('new.json')

# Directory containing models and CSV files
csv_dir = '/content/'  # Directory containing CSV files
model_dir = '/content/'  # Directory containing model files

# List all model files in the directory
model_files = [f for f in os.listdir(model_dir) if f.endswith('.pth')]
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

# Load and test each model on each CSV file
input_size = 4  # Number of features in the input data
# input_size = 5  # Number of features in the input data

for csv_file in csv_files:
    csv_file_path = os.path.join(csv_dir, csv_file)
    for model_file in model_files:
        model_path = os.path.join(model_dir, model_file)
        model_name = os.path.splitext(model_file)[0]
        model = load_trained_model(model_path, input_size)
        plot_predicted_vs_real(csv_file_path, model, model_name, mean_x, std_x, mean_y, std_y)


<ipython-input-43-76f96b6bb6b4>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


MAE: 96.4478
MSE: 22682.5312
MAPE: 31.2790%
CSV saved as: predicted_vs_actual_plots/standard_old_without/Trip_7.csv
MAE: 105.4809
MSE: 26164.0137
MAPE: 30.0585%


<ipython-input-43-76f96b6bb6b4>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


CSV saved as: predicted_vs_actual_plots/standard_old_without/Trip_11.csv
MAE: 76.0919
MSE: 16282.9980
MAPE: 30.5405%


<ipython-input-43-76f96b6bb6b4>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


...

CSV saved as: predicted_vs_actual_plots/standard_old_without/Trip_10.csv
MAE: 101.3383
MSE: 24695.8652
MAPE: 26.5394%


<ipython-input-43-76f96b6bb6b4>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


CSV saved as: predicted_vs_actual_plots/standard_old_without/Trip_9.csv
MAE: 106.9917
MSE: 25699.4609
MAPE: 28.7215%


<ipython-input-43-76f96b6bb6b4>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


CSV saved as: predicted_vs_actual_plots/standard_old_without/Trip_1.csv


In [ ]:
!rm -r '/content/predicted_vs_actual_plots'

In [73]:
!zip -r predicted_vs_actual_plots.zip predicted_vs_actual_plots

  adding: predicted_vs_actual_plots/ (stored 0%)
  adding: predicted_vs_actual_plots/normal_old_without/ (stored 0%)
  adding: predicted_vs_actual_plots/normal_old_without/Trip_11_all.png (deflated 9%)
  adding: predicted_vs_actual_plots/normal_old_without/DENA_TC+_MT_BARBOD_clean_0.5.csv (deflated 81%)
  adding: predicted_vs_actual_plots/normal_old_without/Trip_6_all.png (deflated 9%)
  adding: predicted_vs_actual_plots/normal_old_without/Trip_7.csv (deflated 81%)
  adding: predicted_vs_actual_plots/normal_old_without/Trip_9_all.png (deflated 9%)
  adding: predicted_vs_actual_plots/normal_old_without/Trip_7_all.png (deflated 9%)
  adding: predicted_vs_actual_plots/normal_old_without/Trip_11.csv (deflated 81%)
  adding: predicted_vs_actual_plots/normal_old_without/Trip_10_all.png (deflated 9%)
  adding: predicted_vs_actual_plots/normal_old_without/Trip_14.csv (deflated 82%)
  adding: predicted_vs_actual_plots/normal_old_without/DENA_TC+_MT_BARBOD_clean_0.5_all.png (deflated 9%)
  addin

# **nedc normal**

In [72]:
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn as nn
import torch

# Constants
SEQUENCE_LENGTH = 600  # Updated sequence length
PLOT_SAVE_DIR = 'predicted_vs_actual_plots'  # Base directory to save plots and CSVs
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure the base save directory exists
os.makedirs(PLOT_SAVE_DIR, exist_ok=True)

# Define the PyTorch model structure (same as the one used for training)


# Load the trained model
def load_trained_model(model_path, input_size):
    model = FuelConsumptionModel(input_size=input_size)
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()
    return model

# Process the file and prepare segments
def process_file(file_path):
    df = pd.read_csv(file_path)
    # df = df.iloc[::2].reset_index(drop=True)
    df['Trip fuel consumption'] = df['sys_trip_fc\CCP:1'] - df['sys_trip_fc\CCP:1'].iloc[0]
    df['Acceleration'] = df['sys_veh_spd\CCP:1'].diff().fillna(0)
    df['Momentary fuel consumption'] = df['Trip fuel consumption'].diff().fillna(0)
    df['slope'] = 0
    # Selecting features and target
    features = df[['trs_mt_gr_st\CCP:1', 'sys_veh_spd\CCP:1', 'slope', 'Acceleration','sys_ect\CCP:1']]
    # features = df[['trs_mt_gr_st\CCP:1', 'sys_veh_spd\CCP:1', 'slope', 'Acceleration']]

    target = df['Momentary fuel consumption']
    target = df['Momentary fuel consumption']
    return features, target

# Pad and normalize the data
def pad_and_normalize(data, sequence_length=SEQUENCE_LENGTH):
    padded_data = np.zeros((len(data), sequence_length, data[0].shape[1]))
    for i, seq in enumerate(data):
        length = min(len(seq), sequence_length)
        padded_data[i, :length] = seq[:length]

    # Normalization (same as in your script)
    min_val_x = [0, 0, -10, -10,0]
    max_val_x = [6, 150, 10, 10,130]
    for i in range(padded_data.shape[-1]):
        padded_data[:, :, i] = (padded_data[:, :, i] - min_val_x[i]) / (max_val_x[i] - min_val_x[i])

    return torch.tensor(padded_data, dtype=torch.float32).to(DEVICE)




# Predict and plot the results
def plot_predicted_vs_real(input_file, model, model_name):
    features, actual_values = process_file(input_file)
    num_segments = len(features) // SEQUENCE_LENGTH
    predictions = []

    for i in range(num_segments):
        segment = features.iloc[i * SEQUENCE_LENGTH:(i + 1) * SEQUENCE_LENGTH]
        segment_normalized = pad_and_normalize([segment.values])
        with torch.no_grad():
            segment_predictions = model(segment_normalized).cpu().numpy()
        predictions.extend(segment_predictions.flatten() * 10000)

    # Handle any remaining data
    remainder = len(features) % SEQUENCE_LENGTH
    if remainder != 0:
        last_segment = features.iloc[-remainder:]
        last_segment_normalized = pad_and_normalize([last_segment.values], sequence_length=remainder)
        with torch.no_grad():
            last_segment_predictions = model(last_segment_normalized).cpu().numpy()
        predictions.extend(last_segment_predictions.flatten() * 10000)

    predictions = np.array(predictions)
    actual_valuess = actual_values.values[:len(predictions)]  # Ensure lengths match

    # Calculate error metrics
    mae = mean_absolute_error(actual_valuess, predictions)
    mse = mean_squared_error(actual_valuess, predictions)

    # Handling MAPE: Ignore zero actual values to avoid NaN issues
    non_zero_actual = actual_valuess != 0
    mape = np.mean(np.abs((actual_valuess[non_zero_actual] - predictions[non_zero_actual]) / actual_valuess[non_zero_actual])) * 100

    print(f'MAE: {mae:.4f}---------------{input_file}')
    print(f'MSE: {mse:.4f}')
    print(f'MAPE: {mape:.4f}% ---------------------')

    plt.figure(figsize=(10, 6))
    plt.plot(np.cumsum(actual_values.values[:len(predictions)], axis=0), label='Real', color='blue')
    plt.plot(np.cumsum(predictions[:len(actual_values)], axis=0), label='Predicted', color='red')
    plt.xlabel('Index')
    plt.ylabel('Fuel Consumption')
    plt.title(f'Predicted vs Real Fuel Consumption ({model_name})')
    plt.legend()

    # Create model-specific directory
    model_save_dir = os.path.join(PLOT_SAVE_DIR, model_name)
    os.makedirs(model_save_dir, exist_ok=True)

    # Save plot using model name in the designated directory
    plot_filename = os.path.join(model_save_dir, f'{os.path.splitext(os.path.basename(input_file))[0]}_all.png')
    plt.savefig(plot_filename)
    plt.close()

    # Save predictions and actual values to CSV using model name in the designated directory
    results_df = pd.DataFrame({
        'Speed': features["sys_veh_spd\CCP:1"].iloc[:len(predictions)],
        'Actual': np.cumsum(actual_values.values[:len(predictions)], axis=0),
        'Predicted': np.cumsum(predictions[:len(actual_values)], axis=0),
        'MAE': mae,  # Add MAE to the dataframe
        'MSE': mse,  # Add MSE to the dataframe
        'MAPE': mape  # Add MAPE to the dataframe
    })

    csv_filename = os.path.join(model_save_dir, f'{os.path.splitext(os.path.basename(input_file))[0]}.csv')
    results_df.to_csv(csv_filename, index=False)
    print(f"CSV saved as: {csv_filename}")



# Directory containing all saved models and CSV files
csv_dir = '/content/sample_data'  # Directory containing CSV files
model_dir = '/content/'  # Directory containing model files

# List all model files in the directory
model_files = [f for f in os.listdir(model_dir) if f.endswith('.pth')]

# List all CSV files in the directory
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

# Load and test each model on each CSV file
input_size = 4  # Number of features in the input data
input_size = 5  # Number of features in the input data


for csv_file in csv_files:
    csv_file_path = os.path.join(csv_dir, csv_file)
    # print(f"Processing CSV file: {csv_file}")

    for model_file in model_files:
        model_path = os.path.join(model_dir, model_file)
        model_name = os.path.splitext(model_file)[0]  # Get the base name of the model file
        # print(f"Testing model: {model_name} on {csv_file}")

        model = load_trained_model(model_path, input_size)
        plot_predicted_vs_real(csv_file_path, model, model_name)


<ipython-input-72-0f4df8266c11>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


MAE: 110.2381---------------/content/sample_data/DENA_TC+_MT_BARBOD_clean_0.5.csv
MSE: 24764.8615
MAPE: 59.5199% ---------------------
CSV saved as: predicted_vs_actual_plots/old_normal_withcool/DENA_TC+_MT_BARBOD_clean_0.5.csv


<ipython-input-72-0f4df8266c11>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


KeyError: 'sys_trip_fc\\CCP:1'

In [53]:
df = pd.read_csv('/content/sample_data/DENA_TC+_MT_BARBOD_clean_0.5.csv')

In [54]:
df.columns

Index(['time', 'sys_ect\CCP:1', 'hw_bat_vlt\CCP:1', 'sys_bst_t\CCP:1',
       'trs_mt_gr_st\CCP:1', 'sys_veh_spd\CCP:1', 'fmsp_af_sp\CCP:1',
       'sys_map\CCP:1', 'sys_pps_pct\CCP:1', 'tq_clch_torq\CCP:1',
       'sys_tps_pct\CCP:1', 'sys_bst_p\CCP:1', 'afm_stk_flw\CCP:1',
       'afm_stk_flw_pct\CCP:1', 'tq_imep_ref\CCP:1', 'sys_fc_avg_rate\CCP:1',
       'sys_eng_spd\CCP:1', 'sys_trip_fc\CCP:1', 'knck_ron_fac\CCP:1',
       'sys_iat\CCP:1'],
      dtype='object')

# **Test standard**

In [42]:
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn as nn
import torch

# Constants
SEQUENCE_LENGTH = 600  # Updated sequence length
PLOT_SAVE_DIR = 'predicted_vs_actual_plots'  # Base directory to save plots and CSVs
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure the base save directory exists
os.makedirs(PLOT_SAVE_DIR, exist_ok=True)

# Define the PyTorch model structure (same as the one used for training)


# Load the trained model
def load_trained_model(model_path, input_size):
    model = FuelConsumptionModel(input_size=input_size)
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()
    return model


# Load standardization parameters from JSON
def load_normalization_params(json_file):
    with open(json_file, 'r') as f:
        params = json.load(f)
    mean_x = torch.tensor(params['mean_x'], dtype=torch.float32).to(DEVICE)
    std_x = torch.tensor(params['std_x'], dtype=torch.float32).to(DEVICE)
    mean_y = torch.tensor(params['mean_y'], dtype=torch.float32).to(DEVICE)
    std_y = torch.tensor(params['std_y'], dtype=torch.float32).to(DEVICE)
    return mean_x, std_x, mean_y, std_y

# Custom standardization function for X
def custom_standardize_X(data, mean_vals, std_vals):
    for i in range(data.shape[-1]):
        data[:, :, i] = (data[:, :, i] - mean_vals[i]) / std_vals[i]
    return data

# Custom standardization function for y
def custom_standardize_y(data, mean_val, std_val):
    return (data - mean_val) / std_val

# Process the file and standardize the data
def process_file(file_path, mean_x, std_x, mean_y, std_y):
    df = pd.read_csv(file_path)
    df['Trip fuel consumption'] = df['sys_trip_fc\CCP:1'] - df['sys_trip_fc\CCP:1'].iloc[0]
    df['Acceleration'] = df['sys_veh_spd\CCP:1'].diff().fillna(0)
    df['Momentary fuel consumption'] = df['Trip fuel consumption'].diff().fillna(0)
    df['slope'] = 0
    # Selecting features and target
    features = df[['trs_mt_gr_st\CCP:1', 'sys_veh_spd\CCP:1', 'slope', 'Acceleration','sys_ect\CCP:1']]
    features = df[['trs_mt_gr_st\CCP:1', 'sys_veh_spd\CCP:1', 'slope', 'Acceleration']]

    target = df['Momentary fuel consumption']
    target = df['Momentary fuel consumption']

    # Convert features and target to tensors
    features = torch.tensor(features.values, dtype=torch.float32).to(DEVICE)
    target = torch.tensor(target.values, dtype=torch.float32).to(DEVICE)

    # Apply standardization
    features = custom_standardize_X(features.unsqueeze(0), mean_x, std_x).squeeze(0)
    # target = custom_standardize_y(target, mean_y, std_y)

    return features, target

# Plot and predict function updated to handle standardization
def plot_predicted_vs_real(input_file, model, model_name, mean_x, std_x, mean_y, std_y):
    features, actual_values = process_file(input_file, mean_x, std_x, mean_y, std_y)
    num_segments = len(features) // SEQUENCE_LENGTH
    predictions = []

    for i in range(num_segments):
        segment = features[i * SEQUENCE_LENGTH:(i + 1) * SEQUENCE_LENGTH]
        segment = segment.unsqueeze(0).to(DEVICE)  # Add batch dimension
        with torch.no_grad():
            segment_predictions = model(segment).cpu().numpy()
        predictions.extend(segment_predictions.flatten() * std_y.cpu().numpy() + mean_y.cpu().numpy())  # De-standardize

    remainder = len(features) % SEQUENCE_LENGTH
    if remainder != 0:
        last_segment = features[-remainder:]
        last_segment = last_segment.unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            last_segment_predictions = model(last_segment).cpu().numpy()
        predictions.extend(last_segment_predictions.flatten() * std_y.cpu().numpy() + mean_y.cpu().numpy())  # De-standardize

    predictions = np.array(predictions)
    actual_values = actual_values.cpu().numpy()

    # Calculate error metrics
    mae = mean_absolute_error(actual_values[:len(predictions)], predictions)
    mse = mean_squared_error(actual_values[:len(predictions)], predictions)
    mape = np.mean(np.abs((actual_values[actual_values != 0] - predictions[actual_values != 0]) / actual_values[actual_values != 0])) * 100

    print(f'MAE: {mae:.4f}')
    print(f'MSE: {mse:.4f}')
    print(f'MAPE: {mape:.4f}%')

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(np.cumsum(actual_values[:len(predictions)], axis=0), label='Real', color='blue')
    plt.plot(np.cumsum(predictions, axis=0), label='Predicted', color='red')
    plt.xlabel('Index')
    plt.ylabel('Fuel Consumption')
    plt.title(f'Predicted vs Real Fuel Consumption ({model_name})')
    plt.legend()

    # Save plot and CSV
    model_save_dir = os.path.join(PLOT_SAVE_DIR, model_name)
    os.makedirs(model_save_dir, exist_ok=True)
    plot_filename = os.path.join(model_save_dir, f'{os.path.splitext(os.path.basename(input_file))[0]}_all.png')
    plt.savefig(plot_filename)
    plt.close()

    # Save predictions and actual values to CSV
    results_df = pd.DataFrame({
        'Actual': np.cumsum(actual_values[:len(predictions)], axis=0),
        'Predicted': np.cumsum(predictions, axis=0),
        'MAE': mae,
        'MSE': mse,
        'MAPE': mape
    })
    csv_filename = os.path.join(model_save_dir, f'{os.path.splitext(os.path.basename(input_file))[0]}.csv')
    results_df.to_csv(csv_filename, index=False)
    print(f"CSV saved as: {csv_filename}")

# Load normalization parameters
mean_x, std_x, mean_y, std_y = load_normalization_params('new.json')

# Directory containing models and CSV files
csv_dir = '/content/sample_data'  # Directory containing CSV files
model_dir = '/content'  # Directory containing model files

# List all model files in the directory
model_files = [f for f in os.listdir(model_dir) if f.endswith('.pth')]
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

# Load and test each model on each CSV file
input_size = 4  # Number of features in the input data
# input_size = 5  # Number of features in the input data

for csv_file in csv_files:
    csv_file_path = os.path.join(csv_dir, csv_file)
    for model_file in model_files:
        model_path = os.path.join(model_dir, model_file)
        model_name = os.path.splitext(model_file)[0]
        model = load_trained_model(model_path, input_size)
        plot_predicted_vs_real(csv_file_path, model, model_name, mean_x, std_x, mean_y, std_y)


<ipython-input-42-6a9d1003415f>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


MAE: 86.7235
MSE: 29142.7246
MAPE: 36.4173%
CSV saved as: predicted_vs_actual_plots/standard_old_without/DENA_TC+_MT_BARBOD_clean_0.5.csv


<ipython-input-42-6a9d1003415f>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))


KeyError: 'sys_trip_fc\\CCP:1'